In [1]:
from keras.layers import *
from keras import Model
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras import Sequential
import keras as kr
import keras.losses as losses
 


boat_curved = pd.read_csv("Data/Anomalous_boat_data.csv")
boat_curved= boat_curved.drop(columns=["Unnamed: 0", "heading"])    
scaler = StandardScaler()
anomalous_data = scaler.fit_transform(boat_curved)
print(anomalous_data.shape)







Using TensorFlow backend.


(6620, 7)
